<a href="https://colab.research.google.com/github/newb-dev-1008/Blind-Object-Detection/blob/main/Live_Object_Detection_with_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import os 
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
import matplotlib.pyplot as plt
from PIL import Image

import cv2
cap = cv2.VideoCapture(0)